### Importing the saved csv file

In [9]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.0 MB/s eta 0:

In [10]:
import sqlite3
import pandas as pd
import zipfile
import io
import re
import nltk
import torch
import random
import string
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import chromadb
from chromadb.utils import embedding_functions
from nltk.tokenize import word_tokenize

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path='/content/drive/MyDrive/Enhancing-Search-Engine-Relavance-for-Video-Subtitles/Data/eng_subtitles.csv'

In [6]:
df = pd.read_csv(path)

In [7]:
df.head()

,num,name,content
0,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with opensubtitles free...
1,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,oh i know that its getting late but i dont wan...
2,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,timing and subtitles by the uncontrollable lov...
3,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,apiopensubtitlesorg is deprecated please imple...
4,9408707,battlebots.(2015).eng.1cd,oh no not the minibots oh you leave those litt...


In [13]:
df["content"].fillna("", inplace=True)


<ipython-input-13-f4e624899324>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["content"].fillna("", inplace=True)


### Chunking function

In [8]:
chunk_size  = 500
overlap =100

def chunk_text(text):
    chunks = []
    start =0
    while start < len(text):
        chunk = text[start:start + chunk_size]
        chunks.append(chunk.lower())
        start+=chunk_size -overlap
    return chunks

### Store the embeddings in chromadb


In [9]:
def store_embeddings_in_chromadb(subtitles_df):

    print("Initializing ChromaDB...")
    client = chromadb.PersistentClient(path="/content/drive/MyDrive/Enhancing-Search-Engine-Relavance-for-Video-Subtitles/Data/En_seach_engine_subtitles.db")
    collection = client.get_or_create_collection(name="chromadb_En_sub_embeddings")

    print("Loading embedding model...")
    model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda' if torch.cuda.is_available() else 'cpu')

    print("Cleaning text...")
    subtitles_df['cleaned_content'] = subtitles_df['content']

    batch_size = 100
    print(f"Processing {len(subtitles_df)} subtitles in batches of {batch_size}...")

    for start in range(0, len(subtitles_df), batch_size):
        print(f"Processing batch {start} to {start + batch_size}...")
        end = start + batch_size
        batch = subtitles_df.iloc[start:end]

        all_chunks, all_ids, all_metadatas = [], [], []

        for num, name, content in zip(batch['num'], batch['name'], batch['cleaned_content']):
            chunks = chunk_text(content)
            all_chunks.extend(chunks)
            all_ids.extend([f"{num}_{i}" for i in range(len(chunks))])
            all_metadatas.extend([{"name": name, "content": chunk} for chunk in chunks])

        print(f"Encoding {len(all_chunks)} chunks...")
        with torch.no_grad():
            embeddings = model.encode(all_chunks, batch_size=32, show_progress_bar=True).tolist()

        print("Adding embeddings to ChromaDB...")
        collection.add(
            ids=all_ids,
            embeddings=embeddings,
            metadatas=all_metadatas
        )

    print("Embedding storage complete.")
    return collection

In [10]:
collection = store_embeddings_in_chromadb(df)

Initializing ChromaDB...
Loading embedding model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/204 [00:00<?, ?it/s]

Adding embeddings to ChromaDB...
Processing batch 8300 to 8400...
Encoding 6242 chunks...


Batches:   0%|          | 0/196 [00:00<?, ?it/s]

Adding embeddings to ChromaDB...
Processing batch 8400 to 8500...


TypeError: object of type 'float' has no len()

In [14]:
print(df.isnull().sum())


num                0
name               0
content            0
cleaned_content    1
dtype: int64


In [15]:
# Check how many documents exist in the collection
print(collection.count())


NameError: name 'collection' is not defined

In [6]:
model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
query_text="and about you were doing this interview so i can get to know you and make everything clear to avoid confusion we dont need another mu mu werent you misunderstanding misunderstanding oh mmhmm so why do you want to put up a bar so that i can make monica happy because you love her yeah you love her why what do you like about miss monica"

In [15]:
def search_subtitles(query_text):
    """Reload ChromaDB and search for query embeddings."""
    # Reconnect to ensure fresh data
    chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/Enhancing-Search-Engine-Relavance-for-Video-Subtitles/Data/En_seach_engine_subtitles.db")
    collection = chroma_client.get_or_create_collection(name="chromadb_En_sub_embeddings")

    # Encode query and perform search
    query_embedding = model.encode([query_text])[0].tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=5)

    return results["metadatas"][0] if "metadatas" in results else []


In [16]:
results=search_subtitles(query_text)

In [17]:
print(results)

[]
